
# Make sure to edit the first cell with the appropriate paths !!!!

# Make sure to edit the first cell with the appropriate paths !!!!

In [ ]:
import sys, os
#change the path so that the the directory points to the data folder on your system
data_dir = os.path.expanduser('/Users/kray/Documents/UCD/Crutchfield_Group/ctmc_meps/mothra_run/')


In [ ]:
import numpy as np
import matplotlib.pyplot as plt 

## The cell below pulls data for the first plot, must be run before the next cell

#### read the embedded comments for hints about how to edit what is plotted
#### read the embedded comments for hints about how to edit what is plotted


In [ ]:
all_data = {}
dir_list = sorted(os.listdir(data_dir))
for sdir in [i for i in os.listdir(data_dir) if i[0]!='.']:
    
    sub_dict = {}
    files = [data_dir+sdir+'/'+item for item in os.listdir(data_dir+sdir)]
    data_dict = { f[-6:-4]:np.load(f) for f in files}
    
    for k in data_dict.keys():
        npz_dict = data_dict[k]
        S = npz_dict['s_values']
        N = npz_dict['s_trials']
        
        data = [ npz_dict[f'{s:05}'] for s in S ]
        #for each element, d, of the list "data", d[0] is meps epr, d[1] is ness epr, d[2] is unif epr

        meps_data = [ d[1]-d[0] for d in data]
        '''
        NOTE: meps_data is what the next cell is going to plot on the y axis
        
        if you want to do relative difference, for example,  you could do:
            # meps_data = [ d[1]/d[0]-1 for d in data]
        if you wanted to scale the data differently, like by the total number of possible transitions, you could do something like:
            #meps_data = [ d/s**2-s for d,s in zip(meps_data, S)]
        '''

        #little verification to remove stuff by hand, youll see more on this later
        # should remove a couple of catastorphic failures that were super negative
        meps_data = [ np.abs(d[d>-1E-15]) for d in meps_data]
        
        # nanmean and nanstd shouldn't be doing anything here, but should catch any nan answers that got through my bug checks
        temp_list = [[np.nanmean(item), np.nanstd(item)] for item in meps_data]
        
        #print([ (i[0].shape, i[1].shape) for i in temp_list])
        sub_dict[k] = np.array(temp_list)
        sub_dict[k+'S'] = S
        sub_dict[k+'N'] = N
    all_data[sdir] = sub_dict


In [ ]:

#these three lines are so the color codding is consistent
r_vals = [1,5,10,20,40,80]
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
color_dict = {f'{i:02}':c for i,c in zip(r_vals, colors)}


fig, ax = plt.subplots(1,len(all_data), figsize=(34,5), sharey=True, sharex=True)

k_list = sorted([ int(item) for item in all_data.keys()])

for i,key in enumerate(k_list):
    data = all_data[f'{key:02}']
    for k, val in data.items():
        
        if k[-1] != 'S' and k[-1] != 'N':
            if int(k)>0:
                
                ax[i].errorbar(data[k+'S'], val[:,0], yerr=3*val[:,1]/np.sqrt(data[k+'N']), markersize=4, marker='D',c=color_dict[k], label=f'pump_prcnt={k}')
                #ax[i].legend()
                ax[i].set_xscale('log')  
                ax[i].set_yscale('log') 
                ax[i].set_title(f'pump_strength {key}')
                
ax[-1].legend()

#here you can set manual limits:
ax[0].set_xlim(4.5,11_000)
ax[0].set_ylim(1E-6,20)

ax[0].set_ylabel('EPR diff: $\\left(\\Sigma_{NESS} - \\Sigma_{MEPS}\\right)$')
    

In [ ]:
#fig.savefig(data_dir+'../epr_diff')

# THIS NEXT PLOT SHOWS ALL DATA FOR A PARTICULAR STENGTH VALUE

### good for diagnosing the odd behaving values above

for example you can see the large variance we observe for pump_strength < -20 and pump_prcnt < 5 are probably from some detailed balance machines sneaking in </p>

also, you will note that there is one value for strength=-40, and -80 for which the ness EPR is like -1E112, so that was a huge numerical failure of some sort. this kid of thing can be removed by hand </p>

note that the legend in each plot gives the mnimum and maximum value for EPR difference




In [ ]:
### SET STR VALUE HERE
### SET STR VALUE HERE
### SET STR VALUE HERE

strength = -20

### SET STR VALUE HERE
### SET STR VALUE HERE
### SET STR VALUE HERE

folder = data_dir+f'{strength:02}/'
files = sorted(os.listdir(folder))
J = len(os.listdir(folder))
ax = None

for j, file in enumerate(files):
    ratio = file[-6:-4]
    a = np.load(folder+file)
    S = a['s_values']
    N = a['s_trials']
    
    
    d = [ a[f'{s:05}'] for s in S ]
    meps_data = [ d[0] for d,s in zip(d,S)]
    ness_data = [ d[1] for d,s in zip(d,S)]
    unif_data = [ d[2] for d,s in zip(d,S)]
    

    if j==0 and ax is None:
        fig, ax = plt.subplots(J, len(d), figsize=(30,20), sharex=True, sharey=True)
    
    ax[j,0].set_ylabel('NESS-MEPS;'+f'ratio={ratio:02}')
    for i in range(len(d)):
        if j == 0:
            ax[j,i].set_title(f'S={S[i]}')
        ax[-1,i].set_xlabel('MEPS EPR')

        diff = ness_data[i] - meps_data[i]
        idx = diff < 0
        
        if sum(idx) > 0:
            print(f'found {sum(idx)} instances in {len(idx)} total machines where NESS > MEPS for S={S[i]}, pump_ratio={ratio}')
            diffmin, mepsmin, nessmin = diff[idx].min(), meps_data[i][idx].min(), ness_data[i][idx].min()
            print(f'minimum diff, meps, ness: {diffmin, mepsmin, nessmin}')
        
        kwargs = {'linestyle':'none','marker':'o','markersize':2}
        extreme_vals = f'{np.min(diff):.0E},{np.max(diff):.0E}'
        
        
        #extreme_val = extreme_vals[np.argmax(np.abs(extreme_vals))]
        
        ax[j,i].plot(meps_data[i], diff, **kwargs, alpha=.1, label=extreme_vals)
        
        xmean = meps_data[i].mean()
        ymean = (diff).mean()
        
        linekw = {'linewidth':1, 'linestyle':'--', 'color':'k'}
        ax[j,i].axhline(ymean, **linekw)
        ax[j,i].axvline(xmean, **linekw)
    
        ax[j,i].legend()
               
    
        #ax[j,i].set_yscale('log')

    ax[0,0].set_yscale('log')
    ax[0,0].set_xscale('log')
    ax[0,0].set_xticks([1E-16,1E-8,1E-1, 1E2])
    ax[0,0].set_xlim([1E-20,100])
    fig.suptitle(f'strength = {strength}')

In [ ]:
# note some odd structures appreaing here. I wonder if they are indicative of something real or just the optimization/ minimum finding algortihm...